<a href="https://colab.research.google.com/github/flaviocrispin/dadosempainel/blob/main/notebook_sobre_dados_em_painel_com_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Sincronizar o google colab com o seu google drive
'''
Ao executar a célula, ira abrir uma nova janela onde pedirá sua conta da google e aparecerá uma senha para colar no espaço solicitado!
'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **O que são dados em painel?**

Dados em painel ou dados longitudinais (em inglês panel data) é um termo comum em estatística e suas aplicações em econometria e é utilizado para designar informações de várias unidades amostrais (indivíduos, empresas, etc) acompanhadas, em geral, ao longo do tempo. Dessa forma, as observações são consideradas em duas dimensões uma delas é a unidade amostral e a outra é o tempo. Por exemplo, os preços mensais do quilograma de feijão em vários supermercados durante esse ano. Além disso, o tempo pode ser substituído por outras condições de observação como diferentes lugares.

As informações de cada unidade amostral, composta por todas as suas observações ao longo do tempo é também chamada de séries temporais. Por exemplo, os preços mensais do quilograma de feijão ao longo desse ano em um único supermercado é uma série com 12 observações.

Dados de séries temporais podem ser considerados como um caso particular de dados em painel, pois nesse caso apenas uma unidade amostral é acompanhada ao longo do tempo. Dados em painel são obtidos a partir de estudos longitudinais ou estudos em painel.

Os dados podem ser balanceados quando as observações de todas as unidades estão disponíveis ao longo de todo o período de tempo, ou painéis não-balanceados quando uma ou mais unidades não está presente em todos os períodos da base de dados. O segundo caso pode originar diversos problemas a serem considerados, um deles ocorre se a ausência das unidades na base de dados não é aleatória (exemplo: Atrição). Os modelos de análise mais utilizados são efeitos fixos e efeitos aleatórios. \
Fonte: [Wikipedia](https://en.wikipedia.org/wiki/Panel_analysis) \
Bibliografia complementar \
https://towardsdatascience.com/a-guide-to-panel-data-regression-theoretics-and-implementation-with-python-4c84c5055cf8 \
https://www.statsmodels.org/stable/index.html \
https://pypi.org/project/linearmodels/ \
OBS: OS DADOS PARA ANALISES SE ENCONTRAM EM: \
[DADOS](https://docs.google.com/spreadsheets/d/1YhsP0yTwiW0vU3ypFotFO3SlUFQlBPC8/edit?usp=sharing&ouid=109560462856001090011&rtpof=true&sd=true)


In [ ]:
#@title Carregamento de Instâncias e Importações de bibliotecas
#@markdown Rodar essa célula para carregar as importações:

#@markdown Dependência: LINEAR MODELS e STATS MODEL
!pip install linearmodels &> /dev/null
!pip install pandas &> /dev/null
!pip install numpy &> /dev/null
!pip install scipy &> /dev/null
!pip install patsy &> /dev/null
!pip install statsmodels &> /dev/null


import os

DIR = '/content/drive/MyDrive/Colab Notebooks/mestrado' #@param {type:'string'}
os.chdir(DIR) 

import pandas as pd #biblioteca Pandas
import matplotlib.pyplot as plt #biblioteca Matplotlib
import numpy as np #biblioteca numpy
import statsmodels.formula.api as smf #biblioteca STATSMODEL
from linearmodels import PanelOLS #biblioteca para dados de painel fixo
from linearmodels import PooledOLS #biblioteca de regressão modelo pooled
from linearmodels import RandomEffects #biblioteca de efeitos randomicos

In [ ]:
#@title Carregar arquivo
'''
As colunas ID e YEAR foram transformada em index com o objetivo
de deixar o dataframe em formal de PAINEL
'''
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/mestrado/Base Dados Aula AED2021.xlsx", index_col=[0,1])

In [ ]:
#@title DATAFRAME do arquivo carregado
#@markdown Explicação do dataset \
#@markdown ID - Países\
#@markdown Year - Ano\
#@markdown GDPpc - PIB per cápita \
#@markdown CO2pc - Emissão de CO2 per capita \
#@markdown CH4pc - Emissão de CH4 per capida \
#@markdown EC - Emissão de carbono \
#@markdown x1, x2, x3 = CO2, CH4 e EC respectivamente\
#@markdown lnx1, lnx2, lnx3 = Logarítmo referente a x1, x2 e x3

df.head()

GDPpc       CO2pc      CH4pc  ...          y1         x2          x3
id year                                ...                                   
1  2008  36280  6708.38011  865.41296  ...  6708.38011  865.41296  7151.62935
   2009  34830  6097.28481  854.57303  ...  6097.28481  854.57303  6861.98357
   2010  35390  6454.08410  839.11157  ...  6454.08410  839.11157  7164.36256
   2011  36300  6320.00488  815.92346  ...  6320.00488  815.92346  7139.63037
   2012  36390  5965.30163  798.50644  ...  5965.30163  798.50644  7208.70916

[5 rows x 12 columns]

In [ ]:
#@title Setar variáveis
y1 = df['y1'] #variável dependente/endógena

#Setar variável endógena
endog = y1

#Setar variáveis exógenas
exog = df[['x1', 'x2', 'x3']]

In [ ]:
#@title MODELO OLS (Ordinary Least Square) - Mínimos Quadrados Ordinários
'''
O Método dos Mínimos Quadrados (MMQ), ou Mínimos Quadrados Ordinários (MQO) ou OLS (do inglês Ordinary Least Squares)
é uma técnica de otimização matemática que procura encontrar o melhor ajuste para um conjunto de dados tentando 
minimizar a soma dos quadrados das diferenças entre o valor estimado e os dados observados (tais diferenças são chamadas resíduos)
fonte: wikipedia
'''
results = sm.OLS(endog, exog).fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                     y1   R-squared (uncentered):                   0.861
Model:                            OLS   Adj. R-squared (uncentered):              0.859
Method:                 Least Squares   F-statistic:                              582.5
Date:                Mon, 04 Oct 2021   Prob (F-statistic):                   9.89e-121
Time:                        10:45:22   Log-Likelihood:                         -2674.3
No. Observations:                 286   AIC:                                      5355.
Df Residuals:                     283   BIC:                                      5366.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [ ]:
#@title REGRESSÃO PARA DADOS DE PAINEL DE EFEITOS FIXOS/FIXED EFFECT MODELS
'''
Existem atributos únicos de indivíduos que não variam com o tempo. Ou seja, os atributos únicos para um determinado
indivíduo i são invariáveis ​​no tempo t. Esses atributos podem ou não estar correlacionados com as variáveis ​​dependentes individuais yi
fonte: wikipedia
'''
mod = PanelOLS.from_formula('y1 ~ x1 + x2 + x3 + EntityEffects', df)
fe_res = mod.fit(cov_type='clustered', cluster_entity=True)
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                     y1   R-squared:                        0.4460
Estimator:                   PanelOLS   R-squared (Between):              0.4390
No. Observations:                 286   R-squared (Within):               0.4460
Date:                Mon, Oct 04 2021   R-squared (Overall):              0.4390
Time:                        10:56:15   Log-likelihood                   -2198.4
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      68.970
Entities:                          26   P-value                           0.0000
Avg Obs:                       11.000   Distribution:                   F(3,257)
Min Obs:                       11.000                                           
Max Obs:                       11.000   F-statistic (robust):             14.410
                            

In [ ]:
#@title REGRESSÃO COM DADOS EM PAINEL DE EFEITOS RANDOMICOS/RANDOM EFFECTS
'''
Existem atributos únicos e constantes de tempo de indivíduos que não 
estão correlacionados com os regressores individuais. OLS agrupado 
pode ser usado para derivar estimativas imparciais e consistentes
de parâmetros, mesmo quando atributos de constante de tempo 
estão presentes, mas os efeitos aleatórios serão mais eficientes
fonte: wikipedia
'''


# random effects model
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit() 
#print results
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                     y1   R-squared:                        0.5195
Estimator:              RandomEffects   R-squared (Between):              0.8030
No. Observations:                 286   R-squared (Within):               0.4151
Date:                Mon, Oct 04 2021   R-squared (Overall):              0.7995
Time:                        10:51:25   Log-likelihood                   -2222.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      101.99
Entities:                          26   P-value                           0.0000
Avg Obs:                       11.000   Distribution:                   F(3,283)
Min Obs:                       11.000                                           
Max Obs:                       11.000   F-statistic (robust):             101.99
                            

In [ ]:
#@title TESTE DE HAUSMAN

import numpy.linalg as la
from scipy import stats

def hausman(fe, re):

  b = fe.params
  B = re.params
  v_b = fe.cov
  v_B = re.cov
  df = b[np.abs(b) < 1e8].size
  chi2 = np.dot((b - B).T, la.inv(v_b - v_B).dot(b - B)) 
  
  pval = stats.chi2.sf(chi2, df)
  return chi2, df, pval
hausman_results = hausman(fe_res, re_res) 
print('chi-Squared: ' + str(hausman_results[0]))
print('degrees of freedom: ' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

chi-Squared: 3.9575018126825077
degrees of freedom: 3
p-Value: 0.2660898848014861


In [ ]:
# Perform PooledOLS
from linearmodels import PooledOLS

mod = PooledOLS(endog, exog)
pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)
# Store values for checking homoskedasticity graphically
fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
residuals_pooled_OLS = pooledOLS_res.resids   

In [ ]:
# 3A. Homoskedasticity
import matplotlib.pyplot as plt
 # 3A.1 Residuals-Plot for growing Variance Detection
fig, ax = plt.subplots()
ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS, color = 'blue')
ax.axhline(0, color = 'r', ls = '--')
ax.set_xlabel('Predicted Values', fontsize = 15)
ax.set_ylabel('Residuals', fontsize = 15)
ax.set_title('Homoskedasticity Test', fontsize = 30)
plt.show()

In [ ]:
# 3A.2 White-Test
from statsmodels.stats.diagnostic import het_white, het_breuschpagan
pooled_OLS_dataset = pd.concat([dataset, residuals_pooled_OLS], axis=1)
pooled_OLS_dataset = pooled_OLS_dataset.drop(['year'], axis = 1).fillna(0)
exog = sm.tools.tools.add_constant(dataset['y1']).fillna(0)
white_test_results = het_white(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, white_test_results)))
# 3A.3 Breusch-Pagan-Test
breusch_pagan_test_results = het_breuschpagan(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, breusch_pagan_test_results)))

In [ ]:
# 3.B Non-Autocorrelation
# Durbin-Watson-Test
from statsmodels.stats.stattools import durbin_watson

durbin_watson_test_results = durbin_watson(pooled_OLS_dataset['residual']) 
print(durbin_watson_test_results)